<a href="https://colab.research.google.com/github/mehrshadhosseini28/digikala_quera_2020/blob/master/Digikala_P2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
with zipfile.ZipFile('/content/nlp_comments_users.zip', 'r') as zip_ref:
    zip_ref.extractall("/content/")

In [ ]:
import pandas as pd
import numpy as np
import datetime
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split


import numpy as np
import time
from keras.models import Model
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.preprocessing.text import Tokenizer, one_hot
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt

def my_str(word):
    s=str(word)
    if s=='nan':
        return '.'
    return word
my_dict = {0:' ',1:'Buyer '}

In [ ]:
data = pd.read_csv('/content/drive/My Drive/Digikala/P2/train_users.csv')


advantages = list(map(my_str,data['advantages'].to_list()))
disadvantages = list(map(my_str,data['disadvantages'].to_list()))
comment = list(map(my_str,data['comment'].to_list()))
is_buyer = data['is_buyer'].astype(int)

X = [my_dict[is_buyer[i]] + comment[i] + advantages[i] + disadvantages[i] for i in range(len(comment))]

y = data['verification_status']
y_P3 = data['rate']


In [ ]:
data = pd.read_csv('/content/drive/My Drive/Digikala/P2/test_users.csv')

advantages = list(map(my_str,data['advantages'].to_list()))
disadvantages = list(map(my_str,data['disadvantages'].to_list()))
comment = list(map(my_str,data['comment'].to_list()))
is_buyer = data['is_buyer'].astype(int)

X_t = [my_dict[is_buyer[i]] + comment[i] + advantages[i] + disadvantages[i] for i in range(len(comment))]

In [ ]:
RANDOM_STATE = 42
text_train = X
text_test = X_t
y_train = y
# text_train, text_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=RANDOM_STATE)


tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)

X_train = tokenizer.texts_to_sequences(text_train)
X_test = tokenizer.texts_to_sequences(text_test)

vocab_size = len(tokenizer.word_index) + 1
maxlen = max(len(x) for x in X_train)


X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [ ]:
# Define CNN architecture

embedding_dim = 32

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
print(model.summary())

# Fit model
history = model.fit(X_train, y_train,
                    epochs=4,
                    verbose=True,
                    validation_split=0.05)
loss, accuracy = model.evaluate(X_train, y_train, verbose=True)
print("Training Accuracy: {:.4f}".format(accuracy))
# loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
# print("Testing Accuracy:  {:.4f}".format(accuracy))

In [ ]:
y_p=model.predict(X_test)

In [ ]:
y_p = np.round(y_p)

In [ ]:
id = ['id']
for i in range(len(y_p)):
  id.append(i)

vs = ['verification_status']
for item in y_p:
  vs.append(item)

In [ ]:
import csv

with open('/content/A2.csv', mode='w') as file:
    file_writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    file_writer.writerow([id[0],vs[0]])
    for i,j in zip(id[1:],vs[1:]):
      file_writer.writerow([str(int(i)),str(int(j))])